In [13]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [14]:
cd /content/drive/MyDrive/VinBigData/BML/Project/BML

/content/drive/MyDrive/VinBigData/BML/Project/BML


In [15]:
!pip install --upgrade insightface

Requirement already up-to-date: insightface in /usr/local/lib/python3.6/dist-packages (0.1.5)


In [16]:
!pip install mxnet-mkl

In [17]:
import insightface
import urllib
import urllib.request
import cv2
import numpy as np

def url_to_image(url):
    resp = urllib.request.urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    return image

In [18]:
model = insightface.app.FaceAnalysis()

In [19]:
ctx_id = -1

In [20]:
model.prepare(ctx_id = ctx_id, nms=0.4)

[32, 16, 8] {'32': {'SCALES': (32, 16), 'BASE_SIZE': 16, 'RATIOS': (1.0,), 'ALLOWED_BORDER': 9999}, '16': {'SCALES': (8, 4), 'BASE_SIZE': 16, 'RATIOS': (1.0,), 'ALLOWED_BORDER': 9999}, '8': {'SCALES': (2, 1), 'BASE_SIZE': 16, 'RATIOS': (1.0,), 'ALLOWED_BORDER': 9999}}
use_landmarks True


In [21]:
img =  cv2.imread("/content/drive/MyDrive/VinBigData/BML/Project/BML/images/ameliezilber/2020-11-17_19-03-42_UTC.jpg")

In [22]:
import os
faces = model.get(img)
for idx, face in enumerate(faces):
    print("Face [%d]:"%idx)
    print("\tage:%d"%(face.age))
    gender = 'Male'
    if face.gender==0:
        gender = 'Female'
    print("\tgender:%s"%(gender))
    print("\tembedding shape:%s"%face.embedding.shape)
    print("\tbbox:%s"%(face.bbox.astype(np.int).flatten()))
    print("\tlandmark:%s"%(face.landmark.astype(np.int).flatten()))
    print("")

Face [0]:
	age:25
	gender:Female
	embedding shape:512
	bbox:[345 184 698 642]
	landmark:[424 387 584 363 506 473 472 554 589 533]



In [23]:
import os

In [24]:
features = []
labels = []
i = -1
for directories in os.walk('./images'):
    if i == -1:
        i+=1
        continue
    for file in directories[2]:
        path = directories[0] + '/' + file
        print(path)
        img =  cv2.imread(path)
        faces = model.get(img)
        try:
          features.append(faces[0].embedding)
          
          labels.append(i)
        except:
          pass
    print(i)
    i+=1
    

./images/baoanh0309/2020-12-04_00-32-54_UTC.jpg
./images/baoanh0309/2020-12-07_06-31-05_UTC.jpg
./images/baoanh0309/2020-12-07_06-32-06_UTC.jpg
./images/baoanh0309/2020-12-16_11-21-49_UTC_3.jpg
./images/baoanh0309/2020-12-16_11-21-49_UTC_2.jpg
./images/baoanh0309/2020-12-17_11-19-36_UTC_2.jpg
./images/baoanh0309/2020-12-17_11-19-36_UTC_3.jpg
./images/baoanh0309/2020-12-17_11-19-36_UTC_1.jpg
./images/baoanh0309/2020-12-18_11-38-16_UTC_1.jpg
./images/baoanh0309/2020-12-18_11-38-16_UTC_2.jpg
0
./images/ameliezilber/2020-11-17_19-03-42_UTC.jpg
./images/ameliezilber/2020-11-22_19-21-13_UTC.jpg
./images/ameliezilber/2020-12-07_19-24-29_UTC.jpg
./images/ameliezilber/2020-12-08_21-36-27_UTC.jpg
1
./images/btranofficial/2020-11-04_17-00-23_UTC_profile_pic.jpg
./images/btranofficial/2020-11-23_13-16-08_UTC.jpg
./images/btranofficial/2020-11-29_10-11-59_UTC.jpg
./images/btranofficial/2020-11-30_06-57-55_UTC.jpg
./images/btranofficial/2020-11-30_14-10-29_UTC_1.jpg
./images/btranofficial/2020-11-30

In [25]:
label_name = []
for i in list(os.walk('./images'))[1:53]:
  label_name.append(i[0][9:])


In [26]:
# np.save('/content/drive/MyDrive/VinBigData/BML/Project/BML/features.npy', features)
# np.save('/content/drive/MyDrive/VinBigData/BML/Project/BML/labels.npy', labels)
# np.save('/content/drive/MyDrive/VinBigData/BML/Project/BML/label_name.npy', label_name)

In [27]:
# x = np.load('/content/drive/MyDrive/VinBigData/BML/Project/BML/features.npy', allow_pickle=False)
# y = np.load('/content/drive/MyDrive/VinBigData/BML/Project/BML/labels.npy')
# label_name = np.load('/content/drive/MyDrive/VinBigData/BML/Project/BML/label_name.npy')
x = features
y = labels

In [28]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
svm = SVC()
SVC(C = 0.1, gamma = 0.9)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25)
svm.fit(x_train, y_train)
svm.score(x_test, y_test)

0.9802631578947368

In [29]:
svm.predict(x_test)

array([51, 10,  2, 21,  7, 51, 30, 27, 50,  0, 24, 54, 54, 40, 31, 36,  6,
       35, 15,  3,  3, 14, 27, 24, 24, 49, 45, 42,  4, 51, 11,  5, 54, 27,
       33, 40, 26, 49,  3, 32, 48, 48, 33, 42, 14, 14, 43, 14, 41, 15, 33,
       46, 19, 45,  0, 30, 45, 27, 32, 29, 14, 28, 12, 51, 28, 43, 48, 27,
       42, 49, 49, 13, 45, 48, 54, 18, 51, 25, 15, 27, 34, 15, 15, 18, 10,
       21, 24,  0, 42,  3, 16, 40, 35, 27, 41,  3, 34, 25, 14, 13, 14, 21,
       33, 42, 15, 27,  0, 51, 42, 54,  6,  6, 25, 29, 29, 44, 43, 48, 14,
       33, 51, 29, 32, 12, 26,  4,  5, 40, 42, 43, 54, 23, 15, 19,  9,  7,
        4, 27, 41,  5, 52, 17, 45, 15, 42, 40, 33,  4, 24, 42, 48, 14])

In [30]:
%%time
label_name[svm.predict([x_test[0]])[0]]

CPU times: user 995 µs, sys: 0 ns, total: 995 µs
Wall time: 989 µs


'tu_hhao'

In [31]:
def recognizer(img_path, typ = 1):
  
  img = url_to_image(img_path)
  # print(type(img))
  if typ:
    img = cv2.resize(img, dsize=(504, 540), interpolation=cv2.INTER_CUBIC)
  face = model.get(img)
  emb = face[0].embedding
  return label_name[svm.predict([emb])[0]]

In [32]:
%%time
img_path = ['https://upload.wikimedia.org/wikipedia/commons/f/fe/Son_Tung_M-TP_1_%282017%29.png',
            'https://ivcdn.vnecdn.net/giaitri/images/web/2020/12/21/mv-chung-ta-cua-hien-tai-1608516586.jpg?w=750&h=450&q=100&dpr=1&fit=crop&s=XvJG9jMhjzUhE752ufb9Jw',
            'https://icdn.dantri.com.vn/zoom/1200_630/2020/07/06/sontungmtp-5-1593996464465.jpeg',
            'https://cdn.tgdd.vn/Files/2020/07/06/1268110/4-_1600x900-800-resize.jpg',
            'https://cdn.vietnammoi.vn/2019/11/27/de-thi-ve-son-tung-mtp-1-15748482469981213385641.jpg',
            # diem my 5
            'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTAeXecPQAtQs_hHt1sVB8JTvlsA0wqoo86Hw&usqp=CAU',
            'https://cdn.24h.com.vn/upload/1-2020/images/2020-01-02/1577976269-461-diem-my-9x-diem-my-9x-14-crop-15463198290651558635023-1577775800-width1000height888.jpg',
            'https://image2.baonghean.vn/w607/Uploaded/2020/tamzshuztnzm/2020_10_08/quakhubathaocuadiemmy9xdoihon100trieulamngucdapbatcomcuachong11602001112408width650height245_oujy.jpg',
            'https://img.giaoduc.net.vn/w700/Uploaded/2020/wpxlzdjwp/2012_06_21/hotgirl-diem-my-giaoduc.net%20(11).jpg',
            'https://photo-baomoi.zadn.vn/w700_r1/2020_03_27_296_34469540/8ed4ae13a7504e0e1741.jpg',
            # hoang yen chibi 10
            'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTSdQinu58Eg0q4WdB2F8zZ6ERD2okiUU3WTQ&usqp=CAU',
            'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRGb4EB6A-K-wxvYRkEcyWagW1-PO3CRbS6Ow&usqp=CAU',
            'https://cdn.pose.com.vn/assets/2018/07/hoangyenchibi812_31042993_1909903412373956_8276358643171983360_n.jpg',
            'https://photo-baomoi.zadn.vn/w700_r1/2020_12_18_119_37373467/dade2c0df34e1a10435f.jpg',
            'https://media-cdn.laodong.vn/Storage/NewsPortal/2018/9/13/630708/1522463265-246-Vua-T.jpg?w=414&h=276&crop=auto&scale=both',
            'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQjG0FZgH2_jxL2bZwBLYIf-X9xxtFuRBqeyg&usqp=CAU',
            'https://avatar-ex-swe.nixcdn.com/singer/cover/2020/01/06/0/b/0/d/1578286624991.jpg',
            'https://xone.fm/wp-content/uploads/2020/11/h0-10.png',
            # tran thanh
            'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTKsBqvGUPPI8HZQD34SmaYdgEoSXIzROfJlw&usqp=CAU'
            ]
print(recognizer(img_path[-1], 0))


IndexError: ignored

In [ ]:
import os
for directories in os.walk('./download/'):
    print(directories)